# Preprocessing personalized and self coded tools and Animation Module (TESTS)

### body, head, arm detector and landmarker.
library used opencv (for camera,frame), mediapipe(a very well written DL model)

In [11]:
import cv2
import mediapipe as mp
import csv
import time


mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=True, 
                                model_complexity=2, 
                                min_detection_confidence=0.5, 
                                min_tracking_confidence=0.5)


cap = cv2.VideoCapture(0)  


landmark_data = []


fps_start_time = time.time()
fps_counter = 0
fps = 0
key_face_landmarks_indices = [1, 33, 263, 61, 291, 199]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
  
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    #holistic DL model prediction
    results = holistic.process(rgb_frame)
    

    #result to landmark array insersion
    frame_landmarks = {}
    
    if results.pose_landmarks:
        frame_landmarks['pose'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.pose_landmarks.landmark]
    
    if results.face_landmarks:
        # Filter and store only the key face landmarks based on the selected indices
        frame_landmarks['face'] = [(landmark.x, landmark.y, landmark.z) for idx, landmark in enumerate(results.face_landmarks.landmark) if idx in key_face_landmarks_indices]
    
    if results.left_hand_landmarks:
        frame_landmarks['left_hand'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.left_hand_landmarks.landmark]
    
    if results.right_hand_landmarks:
        frame_landmarks['right_hand'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.right_hand_landmarks.landmark]
    
    landmark_data.append(frame_landmarks)
    
    #result landmark to screen draw the points 
    if results.pose_landmarks:
        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            cx, cy = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
            cv2.circle(frame, (cx, cy), 3, (0, 255, 0), -1)
    
    if results.face_landmarks:
        for idx, landmark in enumerate(results.face_landmarks.landmark):
            # Only draw key face landmarks
            if idx in key_face_landmarks_indices:
                cx, cy = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (cx, cy), 2, (255, 0, 0), -1)
    
    if results.left_hand_landmarks:
        for idx, landmark in enumerate(results.left_hand_landmarks.landmark):
            cx, cy = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
            cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)
    
    if results.right_hand_landmarks:
        for idx, landmark in enumerate(results.right_hand_landmarks.landmark):
            cx, cy = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
            cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)
    
    # Calculate FPS
    fps_counter += 1
    if fps_counter >= 10:  # Update FPS every 10 frames
        fps_end_time = time.time()
        fps = int(fps_counter / (fps_end_time - fps_start_time))
        fps_counter = 0
        fps_start_time = fps_end_time
    
    
    cv2.putText(frame, f"FPS: {fps}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    
    cv2.imshow('kyamara', frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# after the completetion of  the landmark array will be written to  csv
with open('landmark_data.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    header = ['frame', 'type', 'index', 'x', 'y', 'z']
    csvwriter.writerow(header)
    
    for frame_index, frame_data in enumerate(landmark_data):
        for landmark_type, landmarks in frame_data.items():
            for idx, (x, y, z) in enumerate(landmarks):
                csvwriter.writerow([frame_index, landmark_type, idx, x, y, z])


cap.release()
cv2.destroyAllWindows()


comment: serious FPS problems, tensorflow 2.10 above wont take gpu. 
utilize tensorfolow 2.10 or below library, tensor-gpu , and dont forget to check entire dependency versions before even beginning

### (ALL Video on clip folder process TO Landmarks on csv folder)

In [ ]:
import os
import cv2
import mediapipe as mp
import csv
import time


mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, 
                                model_complexity=2, 
                                min_detection_confidence=0.5, 
                                min_tracking_confidence=0.5)


clips_folder = './clips/'


csv_folder = './csv_files/'


os.makedirs(csv_folder, exist_ok=True)


def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    landmark_data = []
    
    
    fps_start_time = time.time()
    fps_counter = 0
    fps = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        
        results = holistic.process(rgb_frame)
        
        
        frame_landmarks = {}
        if results.pose_landmarks:
            frame_landmarks['pose'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.pose_landmarks.landmark]
        if results.face_landmarks:
            frame_landmarks['face'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.face_landmarks.landmark]
        if results.left_hand_landmarks:
            frame_landmarks['left_hand'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.left_hand_landmarks.landmark]
        if results.right_hand_landmarks:
            frame_landmarks['right_hand'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.right_hand_landmarks.landmark]
        
        landmark_data.append(frame_landmarks)
        

        fps_counter += 1
        if fps_counter >= 10:  
            fps_end_time = time.time()
            fps = int(fps_counter / (fps_end_time - fps_start_time))
            fps_counter = 0
            fps_start_time = fps_end_time
        
    
        cv2.putText(frame, f"FPS: {fps}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
    
        cv2.imshow('Frame', frame)
        
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    
    cap.release()
    cv2.destroyAllWindows()
    
    return landmark_data


for folder in os.listdir(clips_folder):
    folder_path = os.path.join(clips_folder, folder)
    
    
    if os.path.isdir(folder_path):
        video_file = os.path.join(folder_path, f"{folder}.mp4")
        
        
        if os.path.exists(video_file):
            print(f"Processing {video_file}...")
            
            
            landmark_data = process_video(video_file)
            
            
            csv_file = os.path.join(csv_folder, f"{folder}.csv")
            with open(csv_file, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                header = ['frame', 'type', 'index', 'x', 'y', 'z']
                csvwriter.writerow(header)
                
                for frame_index, frame_data in enumerate(landmark_data):
                    for landmark_type, landmarks in frame_data.items():
                        for idx, (x, y, z) in enumerate(landmarks):
                            csvwriter.writerow([frame_index, landmark_type, idx, x, y, z])
            
            print(f"CSV file {folder}.csv created.")
        else:
            print(f"No video file found for {folder}.")
    else:
        print(f"{folder} is not a directory.")

print("Processing completed.")


### Landmark CSv animation by input and folder scan(looped)

In [ ]:
import cv2
import numpy as np
import csv
import os

#  read landmark data from CSV file
def read_landmark_data(csv_file):
    landmark_data = {}
    with open(csv_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  
        for row in csvreader:
            frame = int(row[0])
            landmark_type = row[1]
            index = int(row[2])
            x, y, z = float(row[3]), float(row[4]), float(row[5])
            if frame not in landmark_data:
                landmark_data[frame] = {'pose': [], 'face': [], 'left_hand': [], 'right_hand': []}
            landmark_data[frame][landmark_type].append((x, y, z))
    return landmark_data

#  landmarks and connections with colors
def draw_landmarks(img, landmarks, connections, color):
    for (i, j) in connections:
        if i < len(landmarks) and j < len(landmarks):
            pt1 = (int(landmarks[i][0] * img.shape[1]), int(landmarks[i][1] * img.shape[0]))
            pt2 = (int(landmarks[j][0] * img.shape[1]), int(landmarks[j][1] * img.shape[0]))
            cv2.line(img, pt1, pt2, color, 2)
    for (x, y, z) in landmarks:
        cv2.circle(img, (int(x * img.shape[1]), int(y * img.shape[0])), 3, color, -1)

#  connections for pose and hand landmarks
pose_connections = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8),  # Torso and arms
    (9, 10), (11, 12), (11, 13), (13, 15), (15, 17), (12, 14), (14, 16)  # Legs
]

hand_connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),  # Index finger
    (0, 9), (9, 10), (10, 11), (11, 12),  # Middle finger
    (0, 13), (13, 14), (14, 15), (15, 16),  # Ring finger
    (0, 17), (17, 18), (18, 19), (19, 20)  # Pinky finger
]


pose_color = (0, 255, 0)  
hand_color = (0, 0, 255)  

# Function to display standing still body animation with colors
def display_standing_body(img):
    # pose landmarks
    draw_landmarks(img, [], pose_connections, pose_color)
    #  hand landmarks
    draw_landmarks(img, [], hand_connections, hand_color)

# Function to display animation based on CSV file
def display_animation(word):
    csv_file = get_csv_file(word)
    if os.path.exists(csv_file):
        landmark_data = read_landmark_data(csv_file)
        for frame_index in sorted(landmark_data.keys()):
            frame_landmarks = landmark_data[frame_index]
            img = np.zeros((720, 1280, 3), dtype=np.uint8)  # Create a black image

            if 'pose' in frame_landmarks:
                draw_landmarks(img, frame_landmarks['pose'], pose_connections, pose_color)
            if 'face' in frame_landmarks:
                draw_landmarks(img, frame_landmarks['face'], [], (255, 0, 0))  # Blue for face (if needed)
            if 'left_hand' in frame_landmarks:
                draw_landmarks(img, frame_landmarks['left_hand'], hand_connections, hand_color)
            if 'right_hand' in frame_landmarks:
                draw_landmarks(img, frame_landmarks['right_hand'], hand_connections, hand_color)

            cv2.imshow('Stickman Animation', img)
            if cv2.waitKey(50) & 0xFF == ord('q'):
                break
        cv2.destroyAllWindows()
    else:
        print(f'Animation for "{word}" not found.')

# Function to get CSV file path based on user input
def get_csv_file(word):
    return f'./csv_files/{word.lower()}.csv'

# Main function to continuously prompt for input and display animations
def main():
    standing_body = np.zeros((720, 1280, 3), dtype=np.uint8)  # Initial standing still body image
    display_standing_body(standing_body)

    while True:
        word = input("Enter a word to animate (or press Enter to show standing still body, q to quit): ").strip().lower()
        if word == 'q':
            break
        if word:
            display_animation(word)
            display_standing_body(standing_body)

    cv2.destroyAllWindows()

# Start the program
if __name__ == "__main__":
    main()


Enter a word to animate (or press Enter to show standing still body, q to quit):  aama


### continous String Animation 

In [1]:
import cv2
import numpy as np
import csv
import os

# Read landmark data from CSV file
def read_landmark_data(csv_file):
    landmark_data = {}
    with open(csv_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # Skip header
        for row in csvreader:
            frame = int(row[0])
            landmark_type = row[1]
            index = int(row[2])
            x, y, z = float(row[3]), float(row[4]), float(row[5])
            if frame not in landmark_data:
                landmark_data[frame] = {'pose': [], 'face': [], 'left_hand': [], 'right_hand': []}
            landmark_data[frame][landmark_type].append((x, y, z))
    return landmark_data

# Draw landmarks and connections with colors
def draw_landmarks(img, landmarks, connections, color):
    for (i, j) in connections:
        if i < len(landmarks) and j < len(landmarks):
            pt1 = (int(landmarks[i][0] * img.shape[1]), int(landmarks[i][1] * img.shape[0]))
            pt2 = (int(landmarks[j][0] * img.shape[1]), int(landmarks[j][1] * img.shape[0]))
            cv2.line(img, pt1, pt2, color, 2)
    for (x, y, z) in landmarks:
        cv2.circle(img, (int(x * img.shape[1]), int(y * img.shape[0])), 3, color, -1)

# Connections for pose and hand landmarks
pose_connections = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8),  # Torso and arms
    (9, 10), (11, 12), (11, 13), (13, 15), (15, 17), (12, 14), (14, 16)  # Legs
]

hand_connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),  # Index finger
    (0, 9), (9, 10), (10, 11), (11, 12),  # Middle finger
    (0, 13), (13, 14), (14, 15), (15, 16),  # Ring finger
    (0, 17), (17, 18), (18, 19), (19, 20)  # Pinky finger
]

pose_color = (0, 255, 0)  # Green for pose
hand_color = (0, 0, 255)  # Red for hands

# Function to display standing still body animation with colors
def display_standing_body(img):
    # Draw pose and hand landmarks on the image
    draw_landmarks(img, [], pose_connections, pose_color)  # Draw pose landmarks
    draw_landmarks(img, [], hand_connections, hand_color)  # Draw hand landmarks

# Function to display animation based on CSV file
def display_animation(word):
    csv_file = get_csv_file(word)
    if os.path.exists(csv_file):
        landmark_data = read_landmark_data(csv_file)
        for frame_index in sorted(landmark_data.keys()):
            frame_landmarks = landmark_data[frame_index]
            img = np.zeros((720, 1280, 3), dtype=np.uint8)  # Create a black image

            # Draw pose landmarks if available
            if 'pose' in frame_landmarks:
                draw_landmarks(img, frame_landmarks['pose'], pose_connections, pose_color)

            # Draw face landmarks if available
            if 'face' in frame_landmarks:
                draw_landmarks(img, frame_landmarks['face'], [], (255, 0, 0))  # Blue for face (if needed)

            # Draw left hand landmarks if available
            if 'left_hand' in frame_landmarks:
                draw_landmarks(img, frame_landmarks['left_hand'], hand_connections, hand_color)

            # Draw right hand landmarks if available
            if 'right_hand' in frame_landmarks:
                draw_landmarks(img, frame_landmarks['right_hand'], hand_connections, hand_color)

            # Display the image with animation
            cv2.imshow('Stickman Animation', img)
            if cv2.waitKey(50) & 0xFF == ord('q'):
                break
        cv2.destroyAllWindows()
    else:
        print(f'Animation for "{word}" not found.')

# Function to get CSV file path based on user input
def get_csv_file(word):
    return f'./csv_files/{word.lower()}.csv'

# Main function to continuously prompt for input and display animations
def main():
    standing_body = np.zeros((720, 1280, 3), dtype=np.uint8)  # Initial standing still body image
    display_standing_body(standing_body)

    while True:
        input_string = input("Enter words (separated by spaces) to animate, or press Enter to show standing still body, or 'q' to quit: ").strip().lower()
        if input_string == 'q':
            break
        if input_string:
            words = input_string.split()
            for word in words:
                display_animation(word)
                display_standing_body(standing_body)

    cv2.destroyAllWindows()

# Start the program
if __name__ == "__main__":
    main()


Enter words (separated by spaces) to animate, or press Enter to show standing still body, or 'q' to quit:  aama
Enter words (separated by spaces) to animate, or press Enter to show standing still body, or 'q' to quit:  q


### Landmark CSV to animation (word to animation). 
the word with its already data on the db, will utilize it directly. just we need to pass the word to search the db for its landmark positions data.
comment: dont forget to check dependency versions T_T

In [15]:
import csv
import cv2
import numpy as np

# Function to read landmark data from CSV file
def read_landmark_data(csv_file):
    landmark_data = {}
    with open(csv_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  
        for row in csvreader:
            frame = int(row[0])
            landmark_type = row[1]
            index = int(row[2])
            x, y, z = float(row[3]), float(row[4]), float(row[5])
            if frame not in landmark_data:
                landmark_data[frame] = {'pose': [], 'face': [], 'left_hand': [], 'right_hand': []}
            landmark_data[frame][landmark_type].append((x, y, z))
    return landmark_data

# Function to draw landmarks and connections
def draw_landmarks(img, landmarks, connections, color):
    for (i, j) in connections:
        if i < len(landmarks) and j < len(landmarks):
            pt1 = (int(landmarks[i][0] * img.shape[1]), int(landmarks[i][1] * img.shape[0]))
            pt2 = (int(landmarks[j][0] * img.shape[1]), int(landmarks[j][1] * img.shape[0]))
            cv2.line(img, pt1, pt2, color, 2)
    for (x, y, z) in landmarks:
        cv2.circle(img, (int(x * img.shape[1]), int(y * img.shape[0])), 3, color, -1)

# landmarks nodes to be seen as connected or drawn but not written
pose_connections = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8),  # Torso and arms
    (9, 10), (11, 12), (11, 13), (13, 15), (15, 17), (12, 14), (14, 16)  # Legs
]


hand_connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),  # Index finger
    (0, 9), (9, 10), (10, 11), (11, 12),  # Middle finger
    (0, 13), (13, 14), (14, 15), (15, 16),  # Ring finger
    (0, 17), (17, 18), (18, 19), (19, 20)  # Pinky finger
]


landmark_data = read_landmark_data('landmark_data.csv')


for frame_index in sorted(landmark_data.keys()):
    frame_landmarks = landmark_data[frame_index]
    img = np.zeros((720, 1280, 3), dtype=np.uint8)

    if 'pose' in frame_landmarks:
        draw_landmarks(img, frame_landmarks['pose'], pose_connections, (0, 255, 0))
    if 'face' in frame_landmarks:
        draw_landmarks(img, frame_landmarks['face'], [], (255, 0, 0))  
    if 'left_hand' in frame_landmarks:
        draw_landmarks(img, frame_landmarks['left_hand'], hand_connections, (0, 0, 255))
    if 'right_hand' in frame_landmarks:
        draw_landmarks(img, frame_landmarks['right_hand'], hand_connections, (0, 0, 255))

    cv2.imshow('Stickman Animation', img)
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


### MP4 to CSV files version 2.0 (Final Version)

In [19]:
import os
import cv2
import mediapipe as mp
import csv
import time



mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, 
                                model_complexity=2, 
                                min_detection_confidence=0.5, 
                                min_tracking_confidence=0.5)


key_face_landmarks_indices = [1, 33, 263, 61, 291, 199]

input_folder = './myHero/'


output_folder = './SignsCsv/'


os.makedirs(csv_folder, exist_ok=True)


def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    landmark_data = []
    
    
    fps_start_time = time.time()
    fps_counter = 0
    fps = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        
        results = holistic.process(rgb_frame)
        
        
        frame_landmarks = {}
        if results.pose_landmarks:
            frame_landmarks['pose'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.pose_landmarks.landmark]
        if results.face_landmarks:
            frame_landmarks['face'] = [(landmark.x, landmark.y, landmark.z) for idx, landmark in enumerate(results.face_landmarks.landmark) if idx in key_face_landmarks_indices]
        if results.left_hand_landmarks:
            frame_landmarks['left_hand'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.left_hand_landmarks.landmark]
        if results.right_hand_landmarks:
            frame_landmarks['right_hand'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.right_hand_landmarks.landmark]
        
        landmark_data.append(frame_landmarks)
        
    
        fps_counter += 1
        if fps_counter >= 10:  
            fps_end_time = time.time()
            fps = int(fps_counter / (fps_end_time - fps_start_time))
            fps_counter = 0
            fps_start_time = fps_end_time
        
    
        cv2.putText(frame, f"FPS: {fps}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
    
        cv2.imshow('Frame', frame)
        
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    
    cap.release()
    cv2.destroyAllWindows()
    
    return landmark_data




for root, dirs, files in os.walk(input_folder):
    
    if root != input_folder:  
        
        relative_path = os.path.relpath(root, input_folder)
        
        
        output_subfolder = os.path.join(output_folder, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)  

        
        folder_name = os.path.basename(root)

        
        counter = 1

        
        for file in files:
            if file.endswith('.mp4'):  
                video_file = os.path.join(root, file)
                print(f"Processing {video_file}...")

                
                landmark_data = process_video(video_file)

                
                csv_filename = f"{folder_name}{counter}.csv"

                
                csv_file = os.path.join(output_subfolder, csv_filename)

                
                with open(csv_file, 'w', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                    header = ['frame', 'type', 'index', 'x', 'y', 'z']
                    csvwriter.writerow(header)

                    for frame_index, frame_data in enumerate(landmark_data):
                        for landmark_type, landmarks in frame_data.items():
                            for idx, (x, y, z) in enumerate(landmarks):
                                csvwriter.writerow([frame_index, landmark_type, idx, x, y, z])

                print(f"CSV file {csv_filename} created in {output_subfolder}.")

                
                counter += 1

        print(f"Finished processing folder: {relative_path}")

print("Processing completed.")



Processing ./myHero/Achamma\WIN_20241112_17_54_57_Pro.mp4...
CSV file Achamma1.csv created in ./SignsCsv/Achamma.
Processing ./myHero/Achamma\WIN_20241112_17_55_02_Pro.mp4...
CSV file Achamma2.csv created in ./SignsCsv/Achamma.
Processing ./myHero/Achamma\WIN_20241112_17_55_17_Pro.mp4...
CSV file Achamma3.csv created in ./SignsCsv/Achamma.
Processing ./myHero/Achamma\WIN_20241112_17_55_30_Pro.mp4...
CSV file Achamma4.csv created in ./SignsCsv/Achamma.
Processing ./myHero/Achamma\WIN_20241112_17_55_38_Pro.mp4...
CSV file Achamma5.csv created in ./SignsCsv/Achamma.
Processing ./myHero/Achamma\WIN_20241112_17_55_42_Pro.mp4...
CSV file Achamma6.csv created in ./SignsCsv/Achamma.
Processing ./myHero/Achamma\WIN_20241112_17_56_25_Pro.mp4...
CSV file Achamma7.csv created in ./SignsCsv/Achamma.
Processing ./myHero/Achamma\WIN_20241112_17_56_30_Pro.mp4...
CSV file Achamma8.csv created in ./SignsCsv/Achamma.
Processing ./myHero/Achamma\WIN_20241112_17_56_38_Pro.mp4...
CSV file Achamma9.csv creat